In [33]:
import pickle as pickle
import json
import networkx as nx




In [34]:
filepath = '/Users/peterambiel/Desktop/good_model/data_inputs/dicts_with_names.pkl'
with open(filepath, 'rb') as pickle_file:
        input_dict = pickle.load(pickle_file)

In [35]:
hold_params = {'generator_cost': {(i[0], i[2]): d[1] for i, d in input_dict['Plants_Dic'].items()},
                'generator_capacity': input_dict['Plant_capacity_dic'],
                'solar_capex': input_dict['Solar_capital_cost_photov_final'], 
                'solar_CF': {(i[0], i[2], i[3]): d for i,d in input_dict['Solar_capacity_factor_final'].items()}, 
                'solar_max_capacity': input_dict['Solar_regional_capacity_final'],
                'solar_installed_capacity': {i[0]: d for i, d in input_dict['Plant_capacity_dic'].items() if i[1] == 'Solar'},
                'wind_capex': input_dict['Wind_capital_cost_final'], 
                'wind_CF': input_dict['Wind_capacity_factor_final'], 
                'wind_max_capacity': input_dict['Wind_onshore_capacity_final'],
                'wind_installed_capacity': {i: d for i, d in input_dict['Plant_capacity_dic'].items() if i[1] == 'Wind'},
                'wind_transmission_cost': input_dict['Wind_trans_capital_cost_final'],
                'transmission_cost':input_dict['Transmission_Cost_dic'], 
                'transmission_capacity':input_dict['Transmission_Capacity_dic'],
                'enerstor_installed_capacity': {i[0]: d for i, d in input_dict['Plant_capacity_dic'].items() if i[1] == 'EnerStor'},
                'load': input_dict['load_final']}

In [36]:
def recursive_store(d, key, value):
    if len(key) == 1:
        d[key[0]] = value
    else:
        if key[0] not in d:
            d[key[0]] = {}
        recursive_store(d[key[0]], key[1:], value)

def process_dict(input_dict, target_key):
    output_dict = {}
    for key, value in input_dict[target_key].items():
        recursive_store(output_dict, key, value)
    return output_dict
    

In [37]:
# create nodes
output = {}
keys_to_remove = ['solar_installed_capacity', 'enerstor_installed_capacity']

hold = [key for key in hold_params.keys()]
for key in hold:
    if key in keys_to_remove:
        output[key] = hold_params[key]
    else: 
        output[key] = process_dict(hold_params, key)   

In [38]:
# create links
dicts_remove = ['transmission_cost', 'transmission_capacity']
hold_links = {}
hold_nodes = {} 

for key, value in output.items(): 
    if key in dicts_remove: 
        hold_links[key] = value
    else: 
        hold_nodes[key] = value

In [39]:
nodes_dict = {} 
for key, dict in hold_nodes.items():
    for next_key, next_dict in dict.items():
        # Check if next_key exists in test dictionary
        if next_key not in nodes_dict :
            nodes_dict[next_key] = {}
        
        # Assign the value as the dictionary itself, not a set
        nodes_dict[next_key][key] = next_dict

In [ ]:
{id: , dependents: {[type: generator, parameters[{cost: ,capacity:}]]}}

In [84]:
nodes = []
for key, dict in nodes_dict.items(): 
    # key = region 
    # dict = data type
    for sub_key, sub_dict in dict.items(): 
        dependents = []
        if sub_key == 'generator_cost' or sub_key == 'generator_capacity': 
            for next_key, value in sub_dict.items():
                capacity_value = dict['generator_capacity'].get(next_key, 0)
                cost_value =  dict['generator_cost'].get(next_key, 0)
                dependents.append({'id': next_key, 'type': 'generation', 'capacity': capacity_value, 'cost': cost_value})    
        elif sub_key == 'solar_capex': 
            for next_key, next_dict in sub_dict.items():
                # next key = state
                # sub dict = resource class
                for n_key, n_dict in next_dict.items(): 
                    # n key = resource class
                    # n dict = cost class
                    for last_key, last_value in n_dict.items():
                        # last key = cost calss
                        # last value = value
                        dependents.append({'id': 'solar_capex', 'type': 'solar', 'resource_class': n_key, 'cost_class': last_key, 'value': value})
        elif sub_key == 'solar_CF': 
            for next_key, next_dict in sub_dict.items():
                # next key = resource class
                # next dict = hours 
                for n_key, value in next_dict.items(): 
                    # n key = hour 
                    # value = value 
                    dependents.append({'id': 'solar_cf', 'type': 'solar', 'resource_class': next_key, 'hour': n_key, 'value': value})
        elif sub_key == 'solar_installed_capacity': 
            dependents.append({'id': 'installed_capacity', 'type': 'solar', 'value': dict[sub_key]})
        elif sub_key == 'solar_max_capacity': 
            for next_key, next_dict in sub_dict.items():
                    # next key = state
                    # sub dict = resource class
                for n_key, n_dict in next_dict.items(): 
                        # n key = resource class
                        # n dict = cost class
                    for last_key, last_value in n_dict.items():
                            # last key = cost calss
                            # last value = value
                        dependents.append({'id': 'solar_max_capacity', 'type': 'solar', 'resource_class': n_key, 'cost_class': last_key, 'value': value})
        elif sub_key == 'wind_capex': 
            for next_key, next_dict in sub_dict.items():
                # next key = state
                # sub dict = resource class
                for n_key, n_dict in next_dict.items(): 
                    # n key = resource class
                    # n dict = cost class
                    for last_key, last_value in n_dict.items():
                        # last key = cost calss
                        # last value = value
                        dependents.append({'id': 'wind_capex', 'type': 'wind', 'resource_class': n_key, 'cost_class': last_key, 'value': value})
        elif sub_key == 'wind_CF': 
            for next_key, next_dict in sub_dict.items():
                # next key = resource class
                # next dict = hours 
                for n_key, value in next_dict.items(): 
                    # n key = hour 
                    # value = value 
                    dependents.append({'id': 'wind_cf', 'type': 'wind', 'resource_class': next_key, 'hour': n_key, 'value': value})
        elif sub_key == 'wind_max_capacity': 
            for next_key, next_dict in sub_dict.items():
                    # next key = state
                    # sub dict = resource class
                for n_key, n_dict in next_dict.items(): 
                        # n key = resource class
                        # n dict = cost class
                    for last_key, last_value in n_dict.items():
                            # last key = cost calss
                            # last value = value
                        dependents.append({'id': 'wind_max_capacity', 'type': 'wind', 'resource_class': n_key, 'cost_class': last_key, 'value': value})
        elif sub_key == 'wind_installed_capacity': 
            dependents.append({'id': 'wind_installed_capacity', 'type': 'wind', 'value': dict[sub_key].values()})
        elif sub_key == 'wind_transmission_cost': 
            for next_key, next_dict in sub_dict.items():
                    # next key = state
                    # sub dict = resource class
                for n_key, n_dict in next_dict.items(): 
                        # n key = resource class
                        # n dict = cost class
                    for last_key, last_value in n_dict.items():
                            # last key = cost calss
                            # last value = value
                        dependents.append({'id': 'wind_trans_cost', 'type': 'wind', 'resource_class': n_key, 'cost_class': last_key, 'value': value})
        elif sub_key == 'enerstor_installed_capacity': 
            dependents.append({'id': 'storage_capacity', 'type': 'storage', 'value': dict[sub_key]})
        elif sub_key == 'load': 
            for next_key, value in sub_dict.items():
                dependents.append({'id': 'load', 'type': 'load', 'hour': next_key, 'value': value}) 
        nodes.append({'id': key, 'dependents': dependents})

In [61]:
links_dict = {}
for key, dict in hold_links.items():
    for next_key, next_dict in dict.items():
        # Check if next_key exists in test dictionary
        if next_key not in links_dict:
            links_dict[next_key] = {}
        
        # Assign the value as the dictionary itself, not a set
        links_dict[next_key][key] = next_dict

In [62]:
data = output['transmission_capacity']
cost = output['transmission_cost']

links = []

for source, targets in data.items():
    for target, capacity in targets.items():
        item = {
            "source": source,
            "target": target,
            "capacity": capacity, 
            "efficiency": 0.972
        }
        if target in cost[source]:  # Check if cost exists for the relationship
            item["cost"] = cost[source][target]
        links.append(item)

In [80]:
nlg = {'nodes': nodes, 'links': links}

graph = nx.node_link_graph(nlg, directed = True, multigraph = False)

In [ ]:
with open('nodes_dict.json', 'w') as f:
    json.dump(nodes_dict, f)

with open('links_dict.json', 'w') as f: 
    json.dump(links_dict, f)